In [ ]:
!pip3 install autogen-agentchat[graph]~=0.2
!pip3 install python-dotenv==1.0.1
!pip3 install azure-search-documents==11.4.0b8
!pip3 install azure-identity==1.12.0

In [ ]:
!pip install autogen-agentchat~=0.2

In [ ]:
import json
import os

import requests
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv

import autogen
from autogen import AssistantAgent, UserProxyAgent, register_function
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen.cache import Cache

load_dotenv()

# Import Cognitive Search index ENV
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")
AZURE_SEARCH_API_VERSION = os.getenv("AZURE_SEARCH_API_VERSION")
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")

In [ ]:
credential = DefaultAzureCredential()
endpoint = AZURE_SEARCH_SERVICE_ENDPOINT

# from azure.identity import AzureCliCredential
from azure.core.credentials import AzureKeyCredential

# token = credential.get_token("https://cognitiveservices.azure.com/.default")

# print("TOKEN", token.token)

# client = SearchClient(endpoint=endpoint, index_name="vector-balfour", credential=credential)
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))

rec_standards = SearchClient(endpoint=endpoint, index_name="recruitment-industry-rep-index", credential=credential)
job_spec = SearchClient(endpoint=endpoint, index_name="recruitment-jobdesc-index", credential=credential)

In [ ]:
config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o"],
    }
)

# gpt4_config = {
#     "cache_seed": 42,
#     "temperature": 0,
#     "config_list": config_list,
#     "timeout": 120,
# }

config_list_gpt4 = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o"],
    }
)

In [ ]:
config_list_gpt4

In [ ]:
def search_rec_standards(query: str):
    payload = json.dumps(
        {
            "search": query,
            "vectorQueries": [{"kind": "text", "text": query, "k": 5, "fields": "vector"}],
            "queryType": "semantic",
            "semanticConfiguration": "recruitment-industry-rep-index-semantic-configuration",
            "captions": "extractive",
            "answers": "extractive|count-3",
            "queryLanguage": "en-US",
        }
    )

    response = list(rec_standards.search(payload))

    output = []
    for result in response:
        result.pop("text_vector")
        output.append(result)

    return output

In [ ]:
def search_job_spec(query: str):
    payload = json.dumps(
        {
            "search": query,
            "vectorQueries": [{"kind": "text", "text": query, "k": 5, "fields": "vector"}],
            "queryType": "semantic",
            "semanticConfiguration": "recruitment-jobdesc-index-semantic-configuration",
            "captions": "extractive",
            "answers": "extractive|count-3",
            "queryLanguage": "en-US",
        }
    )

    response = list(job_spec.search(payload))

    output = []
    for result in response:
        result.pop("text_vector")
        output.append(result)

    return output

In [ ]:
search_job_spec("what did the GVA amount to")

In [ ]:
recruitmentstandards = AssistantAgent(
    name="BritishRecruitmentReportAnalyser",
    system_message="You are a helpful AI assistant. "
    "You can help with Azure Cognitive Search queries for British Recruitment Industry.",
    llm_config={
        "config_list": config_list_gpt4,
    },
)

jobdesc = AssistantAgent(
    name="JobDescriptionAgent",
    system_message="You are a helpful AI assistant. "
    "You can help with Azure Cognitive Search queries for Job Description Specifications.",
    llm_config={
        "config_list": config_list_gpt4,
    },
)

complianceagent = AssistantAgent(
    name="ComplianceAgent",
    system_message="""Please use the provided Recruitment Industry Report to analyse the Job Description Specification and generate a comprehensive list of evaluation checks. Each check should confirm the specification’s alignment with the recruitment industry standards outlined in the report.

Requirements:

List of Evaluation Checks
Clearly enumerate each check or criterion that verifies the Job Description Specification meets the standards from the Recruitment Industry Report.
Details or Notes (Optional)
If needed, provide brief notes on how each check ensures adherence to best practices.

Output Format (example):
Check: [Description of the evaluation check]
Notes: [Optional clarifications or guidance]

Your structured response will help ensure that the Job Description Specification remains consistent with recruitment industry best practices. Thank you""",
    llm_config={
        "config_list": config_list_gpt4,
    },
)
businessanalyst = AssistantAgent(
    name="BusinessAnalystAgent",
    system_message="""You are a recruitment compliance expert with 30 years of experience creating structured compliance checks for job specification alignment. You have access to all relevant recruitment guidelines and best practices. When you receive a request, you will be directed to an index of reference materials.

In your response, you should supply compliance information in the form of a dictionary containing sections. Each section should be broken down into activities with the keys:
- "activity"
- "controlling_documents"
- "acceptance_criteria"

The response must be a Python dictionary that can be directly parsed (e.g., via ast.literal_eval()), and it should not include any newline (\n) or tab (\t) characters.

The format should be as follows:

{
    "SECTION_HEADING": [
        {
            "activity": "Describe activity",
            "controlling_documents": "List relevant guidelines or references",
            "acceptance_criteria": "Define the conditions under which the activity is deemed compliant"
        }
    ],
    "SECTION_HEADING": [
        {
            "activity": "Describe activity",
            "controlling_documents": "List relevant guidelines or references",
            "acceptance_criteria": "Define the conditions under which the activity is deemed compliant"
        }
    ]
}

For example (note that you would include more sections and more activities in a real scenario):

{
    "1.0 JOB REQUIREMENTS & QUALIFICATIONS": [
        {
            "activity": "1.1 Review Minimum Education Criteria",
            "controlling_documents": "Company Hiring Policy Section 2; Industry Recruitment Standard RS-101",
            "acceptance_criteria": "All candidates must meet or exceed the stated educational background."
        },
        {
            "activity": "1.2 Validate Role-Based Skills",
            "controlling_documents": "Skills Matrix v3; Recruitment Industry Best Practices 2023",
            "acceptance_criteria": "Candidate skill set verified against job description requirements."
        }
    ],
    "2.0 CANDIDATE SELECTION PROCEDURES": [
        {
            "activity": "2.1 Conduct Preliminary Screening",
            "controlling_documents": "Screening Checklist SC-12; Code of Fair Hiring HF-200",
            "acceptance_criteria": "All applicants fairly assessed for basic qualifications before proceeding."
        },
        {
            "activity": "2.2 Schedule and Execute Interviews",
            "controlling_documents": "Interview Framework IF-07; Equal Opportunity Guideline EOG-001",
            "acceptance_criteria": "Interview panel follows standardized questions and scoring to ensure fairness."
        }
    ]
}""",
    llm_config={
        "config_list": config_list_gpt4,
    },
)

# -----------------------------------------
# 3. User Proxy Agent
# -----------------------------------------
user_proxy = UserProxyAgent(
    name="User",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },
    
    is_termination_msg=lambda msg: (
        msg.get("content") is not None and "TERMINATE" in msg["content"]
    ),
    human_input_mode="NEVER"
)

# -----------------------------------------
# 4. Register Correct Functions
# -----------------------------------------
register_function(
    search_rec_standards,
    caller=recruitmentstandards,
    executor=user_proxy,
    name="search_rec_standards",
    description="A tool for searching the Recruitment Industry Report index."
)

register_function(
    search_job_spec,
    caller=jobdesc,
    executor=user_proxy,
    name="search_job_spec",
    description="A tool for searching the Job Description Specification index."
)

# -----------------------------------------
# 5. Example Orchestration Flow
# -----------------------------------------
from autogen import GroupChat, GroupChatManager

# -----------------------------------------
# 1. State Transition Logic
# -----------------------------------------
def state_transition(last_speaker, groupchat):
    """
    Determine the next agent to respond based on the last speaker
    and the context of the conversation.
    """
    messages = groupchat.messages

    if last_speaker is user_proxy:
        # User's question -> Recruitment Standards Analysis
        return recruitmentstandards
    elif last_speaker is recruitmentstandards:
        # Recruitment Standards -> Job Description Analysis
        return jobdesc
    elif last_speaker is jobdesc:
        # Job Description + Recruitment Standards -> Compliance Check
        return complianceagent
    elif last_speaker is complianceagent:
        # Compliance -> Business Analyst for final structured report
        return businessanalyst
    elif last_speaker is businessanalyst:
        # Business Analyst -> Conversation ends
        return None
    else:
        # Fallback if no clear transition
        return None


# -----------------------------------------
# 2. Initialize GroupChat with Agents
# -----------------------------------------
groupchat = GroupChat(
    agents=[user_proxy, recruitmentstandards, jobdesc, complianceagent, businessanalyst],
    messages=[],  # Start with an empty conversation
    max_round=10,  # Limit the number of rounds to prevent infinite loops
    #speaker_selection_method=state_transition,  # Use our state transition logic
)

# -----------------------------------------
# 3. GroupChatManager to Manage Execution
# -----------------------------------------
manager = GroupChatManager(groupchat=groupchat, llm_config={
        "config_list": config_list_gpt4,
    })

# -----------------------------------------
# 4. Initiating the Conversation
# -----------------------------------------
# The user's initial question
user_question = (
    "How does this job specification align with British recruitment industry standards? "
    "Generate a detailed compliance report."
)

# Start the conversation by having the UserProxyAgent send the first message
user_proxy.initiate_chat(manager, message=user_question)

# -----------------------------------------
# 5. Output the Final Messages
# -----------------------------------------
# After the conversation ends, you can access all the messages in `groupchat.messages`.
# for message in groupchat.messages:
#     print(f"{message['sender']}: {message['content']}")


In [ ]:
if __name__ == "__main__":
    import asyncio

    async def main():
        with Cache.disk() as cache:
            await user_proxy.a_initiate_chat(
                itpagent,
                message="Search for 'Tell me about surface pro' in the 'vector-balfour' index",
                cache=cache,
            )

    await main()